In [ ]:
import numpy as np
import os, h5py
import pandas as pd
import variant_effect

In [ ]:
# read df and add strand
all_dfs = []
cagi_data = '/home/shush/profile/QuantPred/datasets/CAGI'
combined_filename = 'combined_cagi.bed'
for filename in os.listdir(cagi_data):
    prefix, regulator = filename.split('.tsv')[0].split('_')

    one_reg = pd.read_csv(os.path.join(cagi_data,filename), skiprows=7, sep='\t', header=None)
    one_reg['regulator'] = regulator
    one_reg['set'] = prefix
    all_dfs.append(one_reg)
    

combined_cagi = pd.concat(all_dfs)
combined_cagi.insert(4, 'strand', '+')
combined_cagi.insert(2,'end',combined_cagi.iloc[:,1]+1)
combined_cagi.iloc[:,0] = 'chr'+combined_cagi.iloc[:,0].astype(str)
combined_cagi.to_csv(combined_filename, sep='\t', header=False, index=None)

In [ ]:
output_filename = 'nonneg_cagi_3K.bed'
expand_range(combined_filename, output_filename)

In [ ]:
fa_filename = 'cagi_3k.fa'
coords_list, seqs_list = variant_effect.convert_bed_to_seq(output_filename, fa_filename, genomefile='/home/shush/genomes/hg19.fa')

In [ ]:
window = 3072
bad_lines = []
N = len(seqs_list)
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None)
mid = window // 2
onehot_ref = []
onehot_alt = []
coord_np = np.empty((N, 4)) # chrom, start, end coordinate array
pos_dict = {'+': 1535, '-':1536}
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand]
#     coord_np[i,3] = pos_dict[strand] - 1535

    if seq[pos] != nonneg_df.iloc[i, 3]:
#         print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        alt = nonneg_df.iloc[i,4]

        onehot = variant_effect.dna_one_hot(seq)
        mutated_onehot = onehot.copy()
        mutated_onehot[pos] = variant_effect.dna_one_hot(alt)[0]
        onehot_ref.append(onehot)

        onehot_alt.append(mutated_onehot) 

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [ ]:
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv('/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/CAGI5/final_cagi_metadata.csv')

In [ ]:
onehot_ref_alt = h5py.File('/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/CAGI5/CAGI_onehot.h5', 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.close()